In [1]:
!conda install -c beautifulsoup4 --yes 


CondaValueError: too few arguments, must supply command line package specs or --file



#### 1. Get the dataframe from the website

In [42]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json

res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
df = pd.read_html(str(table))
my_json=df[0].to_json(orient='records')

df=pd.DataFrame(json.loads(my_json))
df.columns = df.iloc[0]
df = df.iloc[1:]
df.head()

,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront


#### 2. Drop the row with a borough that is "Not assigned"

In [43]:
df=df[df.Borough != 'Not assigned']

#### 3. Set the row that has "Not assigned" neighbourhood with its borough 

In [44]:
df.loc[df.Neighbourhood=='Not assigned','Neighbourhood']=df.loc[df.Neighbourhood=='Not assigned']['Borough'].values[0]

#### 3. Combine neighborhoods with same postal code

In [49]:
df['Neighbourhood'] = df[['Postcode','Borough','Neighbourhood']].groupby(['Postcode'])['Neighbourhood'].transform(lambda x: ','.join(x))
df=df.drop_duplicates()

In [54]:
df=df.reset_index(drop=True)

In [58]:
df[:12]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park,Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor,Lawrence Heigh..."
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge,Malvern,Rouge,Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens,Parkview Hill,Woodbine Garden..."
9,M5B,Downtown Toronto,"Ryerson,Garden District,Ryerson,Garden District"


In [59]:
df.shape

(103, 3)